                                                * Downloading libraries *                                                                   

In [ ]:
!pip3 install nltk rouge-score
!pip3 install torch==2.1.0 torchtext==0.16.0  # Example, use versions compatible with each other
!pip3 install pandas
!pip3 install transformers

In [25]:
! pip3 install sentencepiece

2681.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.2 MB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
!pip3 cache purge

In [10]:
import torch
import torchtext
import sentencepiece as spm
print(torch.__version__)
print(torchtext.__version__)

print("SentencePiece version:", spm.__version__)

/Users/shilpasingh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2.1.0
0.16.0


                                                    Importing libraries                                                                 

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.translate.bleu_score import sentence_bleu
import random
import numpy as np
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer


                                        Creating Sample Dataset                                                                     

In [16]:
# Sample dataset
data = {
    'text': [
        "The quick brown fox jumps over the lazy dog. The fox is very fast and quick, making it hard to catch.",
        "Machine learning is a field of artificial intelligence that focuses on building systems that learn from data. It is widely used in applications like image recognition, speech processing, and recommendation systems.",
        "Python is a popular programming language used for web development, data science, and scripting. Its syntax is simple and easy to learn, which makes it ideal for beginners.",
        "Climate change refers to long-term changes in temperature and weather patterns. It is primarily caused by human activities, especially the burning of fossil fuels.",
        "The human brain is an organ that serves as the center of the nervous system. It controls thoughts, memory, emotions, touch, motor skills, vision, breathing, and more."
    ],
    'summary': [
        "The fox is fast and hard to catch.",
        "Machine learning helps systems learn from data.",
        "Python is a beginner-friendly language.",
        "Climate change is caused by human activities.",
        "The brain controls many bodily functions."
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Show the first few rows
print(df)



                                                text  \
0  The quick brown fox jumps over the lazy dog. T...   
1  Machine learning is a field of artificial inte...   
2  Python is a popular programming language used ...   
3  Climate change refers to long-term changes in ...   
4  The human brain is an organ that serves as the...   

                                           summary  
0               The fox is fast and hard to catch.  
1  Machine learning helps systems learn from data.  
2          Python is a beginner-friendly language.  
3    Climate change is caused by human activities.  
4        The brain controls many bodily functions.  


                                    Exporting the above dataset to csv file                                                     

In [17]:
df.to_csv("text_summary_data.csv",index=False)

In [ ]:
# Loading the dataset from csv
def load_data(file_name):
    df=pd.read_csv(file_name)
    return df


# Function to summarize text using T5
def summarize_text_t5(text, model, tokenizer):
    # Prepare the text for summarization
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate summary ids
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    
    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to load data, summarize and compare
def main():
    
    # Load the dataset from CSV
    #df = load_data('text_summary_data.csv')
    
    
    # Load the pre-trained T5 model and tokenizer
    model_name = "t5-small"  # You can also use "t5-base" or "t5-large" for better performance
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    text = """
    Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from data. 
    These systems are designed to improve their performance over time by identifying patterns in data and making predictions based on those patterns. 
    Machine learning has applications in various fields such as healthcare, finance, and robotics, and it plays a crucial role in the development of autonomous systems.
    """    
    expected_summary="Machine learning is a branch of artificial intelligence and is useful in fields healthcare, finance, and robotics."

        
    # Summarize the text using BART
    predicted_summary = summarize_text_t5(text, model, tokenizer)
        
    # Print the original text, expected summary, and predicted summary
    print(f"Original Text:\n{text}\n")
    print(f"Original Summary:\n{expected_summary}\n")
    print(f"Predicted Summary:\n{predicted_summary}\n")
    
if __name__ == "__main__":
    main()
    